In [5]:
from paddlenlp import Taskflow
import paddle
import pandas as pd
import ast
from datetime import datetime

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
text = ''
files_path = r'../../temp/甘肃省_学习考察.xlsx'

original_data = pd.read_excel(files_path)
data = original_data.copy()

In [6]:
def process_content(x):
    # 防止长度过长
    if len(x) > 1500:
        return x
    return ','.join(x)

data['content'] = data['content'].apply(ast.literal_eval)
data['content'] = data['content'].apply(process_content)


In [7]:
# 关系抽取
schema = {'学习考察触发词': ['考察人', '考察人职位','考察城市'],
          '访问触发词': ['访问人', '访问人职位','访问城市'],
          '学习触发词': ['学习人', '学习人职位','学习城市'],
          '考察触发词': ['考察人', '考察人职位','考察城市'],
          '拜访触发词': ['拜访人', '拜访人职位','拜访城市']
}
ie = Taskflow('information_extraction', schema=schema, batch_size=32)

In [8]:
# schema = {'考察触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '访问触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '学习触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人']
#           }

# 批量处理数据
results = [ie(content) for content in data['content']]

In [9]:
results

In [10]:
# 创建 DataFrame
results_df = pd.DataFrame(results)

current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# 保存为 CSV 文件
output_path = f'{current_time}_extracted_events.csv'
results_df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [12]:
results[0][0]